In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt

In [2]:
right = pd.read_csv('20cm-try2-pilot_test/3-right_leg_tibi_20250429_191121_933.csv', skiprows=10, sep=",")
left = pd.read_csv('20cm-try2-pilot_test/2-left_leg_tibia_20250429_191121_931.csv', skiprows=10, sep=",")
pelvis = pd.read_csv('20cm-try2-pilot_test/1-pelvis(s2)_20250429_191121_924.csv', skiprows=10, sep=",")

In [5]:
right.head()

,PacketCounter,SampleTimeFine,dq_W,dq_X,dq_Y,dq_Z,dv[1],dv[2],dv[3],Status
0,1,1609085621,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
1,2,1609102288,1.0,0.000200,-0.000116,0.000336,0.161353,-0.017578,-0.012474,0
2,3,1609118955,1.0,0.000188,-0.000173,0.000275,0.162614,-0.014434,-0.011841,0
3,4,1609135622,1.0,-0.000142,-0.000163,0.000275,0.162148,-0.015884,-0.011889,0
4,5,1609152289,1.0,0.000030,-0.000241,0.000236,0.162651,-0.016612,-0.011840,0


In [12]:
time = right['SampleTimeFine']-right['SampleTimeFine'].iloc[0]
right['Tim-se']= time*1e-6
right['Tim_dt'] = right['Tim-se'].diff()
right.head()

,PacketCounter,SampleTimeFine,dq_W,dq_X,dq_Y,dq_Z,dv[1],dv[2],dv[3],Status,Time_se,Tim-se,Tim_dt
0,1,1609085621,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1.609086e+09,0.000000,NaN
1,2,1609102288,1.0,0.000200,-0.000116,0.000336,0.161353,-0.017578,-0.012474,0,1.609102e+09,0.016667,0.016667
2,3,1609118955,1.0,0.000188,-0.000173,0.000275,0.162614,-0.014434,-0.011841,0,1.609119e+09,0.033334,0.016667
3,4,1609135622,1.0,-0.000142,-0.000163,0.000275,0.162148,-0.015884,-0.011889,0,1.609136e+09,0.050001,0.016667
4,5,1609152289,1.0,0.000030,-0.000241,0.000236,0.162651,-0.016612,-0.011840,0,1.609152e+09,0.066668,0.016667


In [15]:
right['a_z'] = right['dv[3]'] * 60.0
right.head()

,PacketCounter,SampleTimeFine,dq_W,dq_X,dq_Y,dq_Z,dv[1],dv[2],dv[3],Status,Time_se,Tim-se,Tim_dt,a_z
0,1,1609085621,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1.609086e+09,0.000000,NaN,0.00000
1,2,1609102288,1.0,0.000200,-0.000116,0.000336,0.161353,-0.017578,-0.012474,0,1.609102e+09,0.016667,0.016667,-0.74844
2,3,1609118955,1.0,0.000188,-0.000173,0.000275,0.162614,-0.014434,-0.011841,0,1.609119e+09,0.033334,0.016667,-0.71046
3,4,1609135622,1.0,-0.000142,-0.000163,0.000275,0.162148,-0.015884,-0.011889,0,1.609136e+09,0.050001,0.016667,-0.71334
4,5,1609152289,1.0,0.000030,-0.000241,0.000236,0.162651,-0.016612,-0.011840,0,1.609152e+09,0.066668,0.016667,-0.71040


In [17]:
right['a_z'] = right['dv[3]'] / right['Tim_dt']
right.head()


,PacketCounter,SampleTimeFine,dq_W,dq_X,dq_Y,dq_Z,dv[1],dv[2],dv[3],Status,Time_se,Tim-se,Tim_dt,a_z
0,1,1609085621,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1.609086e+09,0.000000,NaN,NaN
1,2,1609102288,1.0,0.000200,-0.000116,0.000336,0.161353,-0.017578,-0.012474,0,1.609102e+09,0.016667,0.016667,-0.748425
2,3,1609118955,1.0,0.000188,-0.000173,0.000275,0.162614,-0.014434,-0.011841,0,1.609119e+09,0.033334,0.016667,-0.710446
3,4,1609135622,1.0,-0.000142,-0.000163,0.000275,0.162148,-0.015884,-0.011889,0,1.609136e+09,0.050001,0.016667,-0.713326
4,5,1609152289,1.0,0.000030,-0.000241,0.000236,0.162651,-0.016612,-0.011840,0,1.609152e+09,0.066668,0.016667,-0.710386


In [18]:
right['a_z'].describe()

count    479.000000
mean      -1.446206
std        7.013757
min      -98.813284
25%       -1.036329
50%       -0.788444
75%       -0.693556
max       63.784484
Name: a_z, dtype: float64